In [1]:
import time
import pandas as pd
from ortools.linear_solver import pywraplp

In [2]:
AllData = pd.ExcelFile("Input Data.xlsx")
StudentCourseData = pd.read_excel(AllData, 'StudentCourse')
TeacherCourseData = pd.read_excel(AllData, 'TeacherCourse')
TeacherBlockData = pd.read_excel(AllData, 'TeacherDay')
CourseBlockData = pd.read_excel(AllData, 'CourseDay')

print(StudentCourseData)
print("")

print(TeacherCourseData)
print("")

print(TeacherBlockData)
print("")

print(CourseBlockData)

    -  0  1  2  3  4  5  6  7  8  9 10 11
0  S0  Y  Y  Y  -  Y  -  -  -  -  -  -  -
1  S1  -  Y  Y  -  Y  Y  -  -  -  -  -  -
2  S2  -  -  -  Y  -  -  -  -  -  Y  Y  Y
3  S3  Y  Y  -  -  -  -  -  -  Y  -  -  Y
4  S4  -  -  -  -  -  Y  Y  Y  Y  -  -  -
5  S5  -  -  -  Y  -  -  -  Y  -  Y  -  Y
6  S6  Y  -  -  -  -  -  Y  -  -  Y  Y  -
7  S7  -  Y  Y  -  -  Y  Y  -  -  -  -  -

    -  0  1  2  3  4  5  6  7  8  9 10 11
0  T0  Y  -  -  -  -  -  Y  -  Y  -  -  -
1  T1  Y  -  -  -  -  Y  Y  -  Y  -  -  -
2  T2  -  Y  -  Y  -  -  -  Y  -  -  -  -
3  T3  Y  Y  -  -  -  -  -  -  -  -  Y  Y
4  T4  -  -  Y  Y  Y  -  -  -  -  -  -  -
5  T5  -  -  Y  -  -  -  -  Y  Y  Y  -  -

    -  0  1  2  3
0  T0  -  -  N  N
1  T1  -  -  -  -
2  T2  -  -  -  N
3  T3  N  -  -  -
4  T4  -  -  -  -
5  T5  N  N  -  -

      -  0  1  2  3
0    C0  -  -  N  N
1    C1  N  N  -  -
2    C2  -  -  -  -
3    C3  -  -  -  -
4    C4  -  -  -  -
5    C5  -  -  -  -
6    C6  -  -  -  -
7    C7  -  -  -  -
8    C8  -  -  -  -

In [3]:
# StudentCourseData[8][3] ='Y' means that Student S3 wants to take Course C8
print(StudentCourseData[8][3])

# TeacherCourseData[9][5] ='Y' means that Course C5 can be taught by Teacher T9
print(TeacherCourseData[9][5])

# TeacherDayData[1][5] ='N' means that Day D1 cannot be assigned to Teacher T5
print(TeacherBlockData[1][5])

# CourseDayData[3][0] ='N' means that Day D3 cannot be assigned to Course C0
print(CourseBlockData[3][0])


Y
Y
N
N


In [4]:
numStudents = 8
numTeachers = 6
numCourses = 12
numDays = 4

allStudents = range(numStudents)
allTeachers = range(numTeachers)
allCourses = range(numCourses)
allDays = range(numDays)

StudentList=['S0','S1','S2','S3','S4','S5','S6','S7']
TeacherList=['T0','T1','T2','T3','T4','T5']
CourseList=['C0','C1','C2','C3','C4','C5','C6','C7','C8','C9','C10','C11']
DayList=['D0','D1','D2','D3']

In [5]:
solver = pywraplp.Solver('Timetabling Problem', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

start_time = time.time()

# Define our binary variables for the students and teachers
X = {}
for s in allStudents:
    for c in allCourses:
        for d in allDays:
            X[s,c,d] = solver.BoolVar('X[%i,%i,%i]' % (s,c,d))

Y = {}
for t in allTeachers:
    for c in allCourses:
        for d in allDays:
            Y[t,c,d] = solver.BoolVar('Y[%i,%i,%i]' % (t,c,d))
            

            
# Define our objective function
solver.Maximize(solver.Sum(X[s,c,d] for s in allStudents for c in allCourses for d in allDays))



# Each student must take one course on each day
for s in allStudents:
    for d in allDays:
        solver.Add(solver.Sum([X[s,c,d] for c in allCourses]) == 1)  

        
# No teacher may teach more than one course per day
for t in allTeachers:
    for d in allDays:
        solver.Add(solver.Sum([Y[t,c,d] for c in allCourses]) <= 1)  


        
    
current_time = time.time() 
reading_time = current_time - start_time         
sol = solver.Solve()
solving_time = time.time() - current_time

print('Optimization Complete with Total Happiness Score of', round(solver.Objective().Value()))
print("")
print('Our program needed', round(solving_time,3), 
      'seconds to determine the optimal solution')
                

Optimization Complete with Total Happiness Score of 32

Our program needed 0.007 seconds to determine the optimal solution


In [6]:
# Print Output for Students

for s in allStudents:
    for c in allCourses:
        for d in allDays:
            if X[s,c,d].solution_value() == 1:
                print("Student", StudentList[s], "is taking Course", CourseList[c],
                      "on Day", DayList[d])
    print("")


Student S0 is taking Course C0 on Day D0
Student S0 is taking Course C0 on Day D3
Student S0 is taking Course C7 on Day D1
Student S0 is taking Course C7 on Day D2

Student S1 is taking Course C4 on Day D0
Student S1 is taking Course C4 on Day D1
Student S1 is taking Course C5 on Day D3
Student S1 is taking Course C10 on Day D2

Student S2 is taking Course C8 on Day D2
Student S2 is taking Course C10 on Day D0
Student S2 is taking Course C10 on Day D1
Student S2 is taking Course C10 on Day D3

Student S3 is taking Course C0 on Day D2
Student S3 is taking Course C0 on Day D3
Student S3 is taking Course C2 on Day D1
Student S3 is taking Course C7 on Day D0

Student S4 is taking Course C0 on Day D0
Student S4 is taking Course C1 on Day D1
Student S4 is taking Course C2 on Day D2
Student S4 is taking Course C6 on Day D3

Student S5 is taking Course C6 on Day D0
Student S5 is taking Course C6 on Day D2
Student S5 is taking Course C9 on Day D1
Student S5 is taking Course C9 on Day D3

Studen

In [7]:
for t in allTeachers:
    for c in allCourses:
        for d in allDays:
            if Y[t,c,d].solution_value() == 1:
                print("Teacher", TeacherList[t], "is teaching Course", CourseList[c],
                      "on Day", DayList[d])
    print("")

Teacher T0 is teaching Course C1 on Day D3
Teacher T0 is teaching Course C5 on Day D0
Teacher T0 is teaching Course C9 on Day D2

Teacher T1 is teaching Course C3 on Day D0
Teacher T1 is teaching Course C5 on Day D3
Teacher T1 is teaching Course C6 on Day D2
Teacher T1 is teaching Course C11 on Day D1

Teacher T2 is teaching Course C2 on Day D1
Teacher T2 is teaching Course C4 on Day D0
Teacher T2 is teaching Course C9 on Day D2

Teacher T3 is teaching Course C4 on Day D2
Teacher T3 is teaching Course C10 on Day D0
Teacher T3 is teaching Course C10 on Day D1
Teacher T3 is teaching Course C11 on Day D3

Teacher T4 is teaching Course C1 on Day D2
Teacher T4 is teaching Course C6 on Day D3
Teacher T4 is teaching Course C9 on Day D1

Teacher T5 is teaching Course C3 on Day D2
Teacher T5 is teaching Course C5 on Day D3
Teacher T5 is teaching Course C11 on Day D0
Teacher T5 is teaching Course C11 on Day D1

